In [0]:
pip install azure.storage.blob

Python interpreter will be restarted.
  Using cached azure_storage_blob-12.16.0-py3-none-any.whl (387 kB)
  Using cached isodate-0.6.1-py2.py3-none-any.whl (41 kB)
  Using cached azure_core-1.27.1-py3-none-any.whl (174 kB)
  Using cached typing_extensions-4.6.3-py3-none-any.whl (31 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing-extensions 4.1.1
    Not uninstalling typing-extensions at /databricks/python3/lib/python3.9/site-packages, outside environment /local_disk0/.ephemeral_nfs/envs/pythonEnv-b7b1ca1d-9030-4be5-b1ad-a69a4eba63ec
    Can't uninstall 'typing-extensions'. No files were found to uninstall.
Python interpreter will be restarted.


In [0]:
import requests
import pandas as pd
import json
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient
from azure.core.pipeline.transport import HttpResponse
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.functions import from_json
from pyspark.sql.types import StructType
from pyspark.sql import SparkSession
import json
import gzip

In [0]:

def get_data(date, time):

    url = "https://data.gharchive.org/"+ str(date) +'-'+ str(time) +'.json.gz'
    print(url)

    response = requests.get(url)

    if response.status_code == 200:

       with open("cache.json.gz", "wb") as file:

            file.write(response.content)

       print("File downloaded successfully.")

    else:

       print("Error downloading the file.")

In [0]:
def unzip_jsongz():
    input_file = '/databricks/driver/cache.json.gz'
    output_file = '/databricks/driver/cache.json'
    with gzip.open(input_file, 'rb') as gz_file:
        with open(output_file, 'wb') as out_file:
            out_file.write(gz_file.read())

In [0]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

In [0]:
get_data("2023-04-04", 15)
unzip_jsongz()
rdd1 = spark.sparkContext.textFile("file:///databricks/driver/cache.json")
get_data("2023-04-04", 16)
unzip_jsongz()
rdd2 = spark.sparkContext.textFile("file:///databricks/driver/cache.json")
get_data("2023-04-04", 18)
unzip_jsongz()
rdd3 = spark.sparkContext.textFile("file:///databricks/driver/cache.json")
rdd_All = rdd1.union(rdd2).union(rdd3)

https://data.gharchive.org/2023-04-04-15.json.gz
File downloaded successfully.
https://data.gharchive.org/2023-04-04-16.json.gz
File downloaded successfully.
https://data.gharchive.org/2023-04-04-18.json.gz
File downloaded successfully.


In [0]:
columns_to_drop = [2,3,5,6,7]

def drop_columns(row):
    return [value for index, value in enumerate(row) if index not in columns_to_drop]

new_rdd = rdd_All.map(drop_columns)

In [0]:
data = new_rdd.collect()
for element in data:
    print(element)

---------------------------------------------------------------------------
Py4JJavaError                             Traceback (most recent call last)
File <command-3582645876172033>:1
----> 1 data = new_rdd.collect()
      2 for element in data:
      3     print(element)

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/rdd.py:1825, in RDD.collect(self)
   1823 with SCCallSiteSync(self.context):
   1824     assert self.ctx._jvm is not None
-> 1825     sock_info = self.ctx._jvm.PythonRDD.collectAndServe(self._jrdd.rdd())
   1826 return list(_load_from_socket(sock_info, self._jrdd_deserializer))

File /databricks/spark/python/lib

In [0]:
data = rdd.take(10)
for element in data:
    print(element)